In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

In [2]:
with open("../data/train.pkl", "rb") as f:
    train_data = pickle.load(f)

In [3]:
train_data

[(array([ -1.,  -1.,  -1., ...,  78.,  40., 144.]), 0),
 (array([ -1.,  -1., 144., ...,  32.,  -1.,  -1.]), 0),
 (array([ 66., 100., 148., 148., 146.,  64., 146., 148.,  82.,   0.,  82.,
         100.,  34., 132., 180.,  65.,  80.,  81., 131.,  52.,  34.,  52.,
          64.,  52.,   3.,  66., 147.,  20.,   4., 132., 132., 100., 111.,
          74., 110.,  60.,  92.,  65., 100., 189.,  44.,   8.,   5.,  76.,
          31., 159.,   5., 124.,   4.,  12.,  51., 157.,  57.,  31., 183.,
          57.,  65.,  92.,  69., 124., 122.,  79., 110.,  76.,  12.,  12.,
          12.,  12.,   8., 159.,  12., 156., 100., 111.,  52., 121.,  36.,
          47.,  41.,  41.,   8., 172.,  38.,  12.,  78.,  12.,  88.,  47.,
         119.,  20.,  47.,  12., 159.,  20.,  20.,  76.,  60., 110., 132.,
         185., 120.,  45., 110., 110.,  72., 124., 178.,  73.,   8.,  78.,
          12.,  42., 173.,  12., 150.,  84.,  66., 152.,  69.,   8.,  41.,
         159.,   5.,  78.,  44., 180.,  71.,  92., 152.,   6., 

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
df = pd.DataFrame(train_data, columns = ["Sequence", "Author"])

In [6]:
df["Author"].value_counts()

Author
0    1630
1     478
3     441
4     236
2     154
Name: count, dtype: int64

In [7]:
class MusicDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in X]
        self.y = torch.tensor(y, dtype=torch.long)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.y)

In [8]:
scaler = StandardScaler()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df["Sequence"], df["Author"], test_size = 0.2, random_state = 42)

In [10]:
X_train

456     [80.0, 80.0, 80.0, 158.0, 92.0, 92.0, 93.0, 12...
462     [0.0, 47.0, 47.0, 157.0, 28.0, 28.0, 12.0, 0.0...
2216    [-1.0, -1.0, -1.0, 0.0, 12.0, 125.0, 12.0, 12....
2668    [144.0, 144.0, 34.0, 68.0, 20.0, 32.0, 146.0, ...
381     [64.0, 64.0, 92.0, 159.0, 28.0, 47.0, 78.0, 64...
                              ...                        
1638    [-1.0, -1.0, 112.0, 0.0, 0.0, 117.0, 12.0, 12....
1095    [0.0, 82.0, 148.0, 148.0, 132.0, 3.0, 6.0, 8.0...
1130    [-1.0, -1.0, -1.0, 0.0, 5.0, 13.0, 78.0, 12.0,...
1294    [12.0, 12.0, 12.0, 12.0, 47.0, 44.0, 44.0, 44....
860     [-1.0, -1.0, 0.0, 0.0, 0.0, 13.0, 172.0, 92.0,...
Name: Sequence, Length: 2351, dtype: object

In [11]:
train_dataset = pd.concat([X_train, y_train], axis =1)
valid_dataset = pd.concat([X_test, y_test], axis = 1)

In [12]:
train_dataset = MusicDataset(train_dataset.iloc[:,0], train_dataset.iloc[:, 1])

In [13]:
print(type(train_dataset[10]))
print(train_dataset[10])

<class 'tuple'>
(tensor([  2.,  36.,  68., 116., 148.,   1.,  82.,  50.,  84.,  21.,  92., 136.,
         92.,  14.,  28.,   9., 175.,  60., 108.,  13., 173., 175.,  92.,  92.,
        175.,  13.,  60., 126.,  12.,  12.,  14.,  95.,  13., 175., 126.,  92.,
         12., 126.,  92.,   0., 173.,  13.,  28.,  61.,  14.,  60., 136.,  92.,
         92., 175., 108.,  28.,  61.,   9., 108., 125.,  13., 126.,  92.,  68.,
          2.,  65., 180.,  28.,  12.,  12., 126.,  28.,  95.,  61.,  14.,  13.,
         60.,  93.,  60., 126., 175., 111.,  28., 158., 158.,  92.,  93.,  12.,
         95., 140., 175.,  15., 140.,  14.,  92., 159.,  93., 180.,  12., 126.,
         12.,   1.,  46.,  42.,  95.,  95., 140.,  12.,  12.,  12.,  28.,  12.,
         42.,  42.,  12., 140.,  12.,  12., 126.,  28.,  12.,  28.,  60.,  29.,
         15.,  12.,  12.,   8., 125.,  60.,  28.,  60.,  12.,  60.,  95.,  12.,
         12., 185., 119.,  95.,  95.,  56., 140., 108.,  95.,  92.,  36.,   2.,
         68., 140., 140

In [14]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.layer_dim, batch_size, self.hidden_dim)
        cell = torch.zeros(self.layer_dim, batch_size, self.hidden_dim)
        return hidden, cell
    
    def forward(self, x , hidden):
        x = torch.transpose(x, 0, 1)
        outputs, hidden = self.lstm(x, hidden)
        x = self.fc(outputs[-1])
        return x, hidden
           
        

In [15]:
df["Author"].nunique()

5

In [16]:
hidden_dim = 6
layer_dim = 8
input_dim = 1
out_dim = df["Author"].nunique()
model = LSTMClassifier(input_dim, hidden_dim, layer_dim, out_dim)
model.to(device)


LSTMClassifier(
  (lstm): LSTM(1, 6, num_layers=8)
  (fc): Linear(in_features=6, out_features=5, bias=True)
)

In [23]:
batch_size = 64
optimizer = torch.optim.Adam(model.parameters(), 0.001)
loss_fun = nn.CrossEntropyLoss()

In [24]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

# Wyrównanie sekwencji w mini batch
def pad_collate(batch, pad_value=pad):
    xx, yy = zip(*batch)
    x_lens = [len(x) if x.dim() > 0 else 0 for x in xx]
    y_lens = [len(y) if y.dim() > 0 else 0 for y in yy]

    xx = [x.unsqueeze(0) if x.dim() == 0 else x for x in xx]
    yy = [y.unsqueeze(0) if y.dim() == 0 else y for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [25]:
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, collate_fn = pad_collate)
train_dataset[10]

(tensor([  2.,  36.,  68., 116., 148.,   1.,  82.,  50.,  84.,  21.,  92., 136.,
          92.,  14.,  28.,   9., 175.,  60., 108.,  13., 173., 175.,  92.,  92.,
         175.,  13.,  60., 126.,  12.,  12.,  14.,  95.,  13., 175., 126.,  92.,
          12., 126.,  92.,   0., 173.,  13.,  28.,  61.,  14.,  60., 136.,  92.,
          92., 175., 108.,  28.,  61.,   9., 108., 125.,  13., 126.,  92.,  68.,
           2.,  65., 180.,  28.,  12.,  12., 126.,  28.,  95.,  61.,  14.,  13.,
          60.,  93.,  60., 126., 175., 111.,  28., 158., 158.,  92.,  93.,  12.,
          95., 140., 175.,  15., 140.,  14.,  92., 159.,  93., 180.,  12., 126.,
          12.,   1.,  46.,  42.,  95.,  95., 140.,  12.,  12.,  12.,  28.,  12.,
          42.,  42.,  12., 140.,  12.,  12., 126.,  28.,  12.,  28.,  60.,  29.,
          15.,  12.,  12.,   8., 125.,  60.,  28.,  60.,  12.,  60.,  95.,  12.,
          12., 185., 119.,  95.,  95.,  56., 140., 108.,  95.,  92.,  36.,   2.,
          68., 140., 140.,  

In [26]:
first_batch = next(iter(train_loader))
print(len(first_batch))

4


In [28]:
EPOCHS_NUMBER = 100
tqdm_progress = tqdm(range(EPOCHS_NUMBER))
model.train()
batch_num = 0
for epoch in tqdm_progress:
    train_losses = 0
    for x, targets, x_len, target_len in train_loader:
        optimizer.zero_grad()
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device).squeeze(1)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(x, (hidden, state))
        preds = preds.squeeze(1)
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        train_losses += loss.item()
    
    tqdm_progress.set_postfix(
        {
            "Epoch": epoch + 1,
            "Los in train": {train_losses},

        }
    )
        
        

  8%|▊         | 8/100 [07:17<1:23:47, 54.64s/it, Epoch=8, Los in train={46.97917425632477}] 


KeyboardInterrupt: 